In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests

In [2]:
# 크롬 웹드라이버 실행 경로
path = 'chromedriver.exe'

# 크롤링할 주소 (네이버 세부설정까지 완료한 상태로)
target_url = "https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2023.01.01&de=2023.12.31docid=&related=0&mynews=1&office_type=1&office_section_code=3&news_office_checked=1009&nso=so%3Ar%2Cp%3Afrom20050101to20050115&is_sug_officeid=0&office_category=0&service_area=0"

# 크롬 드라이버 사용
service = Service(executable_path=path) # selenium 최근 버전은 이렇게
options = webdriver.ChromeOptions()
driver = webdriver.Chrome()

# 드라이버에게 크롤링할 대상 알려주기
try:
    # 웹페이지 열기
    driver.get(target_url)

    # 초기 스크롤 높이 설정
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 아래까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 적응 시간
        time.sleep(0.5)  

        # 새로운 스크롤 높이 계산
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 만약 더 스크롤할게 없다면 정지
        last_height = new_height

    # 해당 화면 안에서 페이지 소스 받기
    page_source = driver.page_source

    # BeautifulSoup으로 parsing
    bs_obj = BeautifulSoup(page_source, 'html.parser')

    # a태그로 info에 해당하는 태그들 다 수집
    results = bs_obj.find_all('a', {'class': "info"})

    # 수집한 a 태그 개수 -> 절반이 네이버 뉴스
    print(f"Number of results found: {len(results)/2}")

finally:
    # 가상 화면 닫기
    driver.quit()

Number of results found: 75.0


In [3]:
# 기존 한국경제 링크랑 섞여서 네이버 뉴스꺼만 필터링
news_link = []
for tag in results:
    href = tag.get('href')
    if href:
        news_link.append(href)

news_link = [i for index, i in enumerate(news_link) if (index + 1) % 2 == 0]

In [4]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [5]:
from tqdm import tqdm

all_results = dict()

# tqdm을 사용하여 진행 상황 표시
for i, link in enumerate(tqdm(news_link, desc="Processing links")):     
    # scraping하려는 웹페이지 주소를 get()에 전달
    driver.get(link)
    all_results[i] = dict()
    
    # 일시 추출하기
    date = driver.find_elements(By.CLASS_NAME, 'media_end_head_info_datestamp_bunch')
    date = date[0].text[2:]
    
    # 제목 추출하기
    title = driver.find_elements(By.CLASS_NAME, 'media_end_head_headline')
    title = title[0].text
    
    # 본문 추출하기
    body = driver.find_elements(By.ID, 'newsct_article')
    body = body[0].text.replace('\n','')
    
    # 결과 저장
    all_results[i]['link'] = link
    all_results[i]['date'] = date
    all_results[i]['title'] = title
    all_results[i]['body'] = body

# 드라이버 종료
driver.quit()

Processing links: 100%|████████████████████████████████████████████████████████████████| 75/75 [01:10<00:00,  1.06it/s]


In [6]:
news_df = pd.DataFrame(all_results).T

In [7]:
# '오전'과 '오후'를 영어로 변환
news_df['date'] = news_df['date'].str.replace('오전', 'AM').str.replace('오후', 'PM')

# date 열을 datetime 형식으로 변환
news_df['date'] = pd.to_datetime(news_df['date'], format='%Y.%m.%d. %p %I:%M')

# year, month, day, hour, minute 열 생성
news_df['year'] = news_df['date'].dt.year
news_df['month'] = news_df['date'].dt.month
news_df['day'] = news_df['date'].dt.day
news_df['hour'] = news_df['date'].dt.hour
news_df['minute'] = news_df['date'].dt.minute

In [8]:
# 고유 인덱스 부여 (년도 + 6자리)
# 1이면 -> 000001
news_df.index = news_df.index + 1
news_df = news_df.reset_index()
news_df['index'] = news_df['index'].astype(str).str.zfill(6)
news_df['index'] = str(news_df['year'].unique()[0]) + news_df['index']

In [9]:
news_df

,index,link,date,title,body,year,month,day,hour,minute
0,2005000001,https://n.news.naver.com/mnews/article/009/000...,2005-01-03 09:38:00,[국내경제 전망] 주력품목 수출전선 '비상등',올해 국내 산업 전망은 그 어느 때보다 어둡다. 지난해 경제를 지탱했던 주력 산업의...,2005,1,3,9,38
1,2005000002,https://n.news.naver.com/mnews/article/009/000...,2005-01-03 09:38:00,"[초일류를 향해] 삼성, 李회장 ""어려울수록 투자 늘려라""",국내 최대 그룹인 삼성은 올 한 해를 성장 잠재력을 키우는 해로 삼을 태세다.200...,2005,1,3,9,38
2,2005000003,https://n.news.naver.com/mnews/article/009/000...,2005-01-03 09:38:00,日 LCD연합전선 결성,"한국 삼성전자와 LG필립스LCD, 일본 샤프 등 세계 LCD 3강을 추격하는 연합전...",2005,1,3,9,38
3,2005000004,https://n.news.naver.com/mnews/article/009/000...,2005-01-03 09:38:00,"""DVD로 로열티 받는 날 온다""",DVD 다음 단계인 차세대 광(光)저장장치 세계표준을 놓고 '블루레이(Blu-Ray...,2005,1,3,9,38
4,2005000005,https://n.news.naver.com/mnews/article/009/000...,2005-01-03 09:38:00,[초일류를 향해] 대기업 '공격경영' 라인업,위기를 기회로 만들어야 한다는 생각은 언제나 경영자 머릿속을 떠나지 않는다 . 그러...,2005,1,3,9,38
...,...,...,...,...,...,...,...,...,...,...
70,2005000071,https://n.news.naver.com/mnews/article/009/000...,2005-01-07 17:11:00,[표] 거래소 외국인 순매수ㆍ순매도 상위종목,(단위＝억원ㆍ만주) ==============================...,2005,1,7,17,11
71,2005000072,https://n.news.naver.com/mnews/article/009/000...,2005-01-07 17:23:00,"""삼성전자에 OLED 못준다""",삼성SDI가 자회사인 삼성OLED 합병을 추진하는 것은 PDP LCD에 이은 차세대...,2005,1,7,17,23
72,2005000073,https://n.news.naver.com/mnews/article/009/000...,2005-01-07 18:02:00,[사설] 美 전자쇼서 올린 기술력 쾌거,지난 6일부터 미국 라스베이거스에서 개최되고 있는 세계 전자쇼에서 우리 기 업 전자...,2005,1,7,18,2
73,2005000074,https://n.news.naver.com/mnews/article/009/000...,2005-01-07 19:53:00,[거래소공시] 남한제지 / 기아자동차 / 삼성전자 등,◇남한제지 = 풍만제지 등 제지3사 합병보도에 대해 합병 관련 확정사항 없다 고 답...,2005,1,7,19,53


In [10]:
news_df.to_csv('2023_매일경제.csv', index=False, encoding='utf-8-sig')